In [ ]:
# target = 'diff_RECEIPT_DATE_PO_FIRST_APPROVED_DATE'

In [1]:
try:
    import pandas as pd
    #from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import KFold
    import datetime as dt
    pd.set_option('display.max_columns',None)
    
    import xgboost as xgb
    

    from sklearn.metrics import r2_score,mean_squared_error
    import math
    import time

    import warnings
    warnings.filterwarnings('ignore')
    
    print('libraries are found')
except:
    print('Some libraries need to be installed')

libraries are found


In [6]:
def read_data(file_name):
    df = pd.read_hdf(file, key='data')
    return df

file = '/workspace/luca/Tabular/e2e/data/data_train_v7.h5'

data = read_data(file)

In [7]:
data.shape

(99563, 88)

In [8]:
## remove columns which contain more than 50% of null values
def treat_null_values(df):
    
    df = df.dropna(thresh=df.shape[0]*0.5, how='all',axis=1) # -3
    df = df.dropna(axis=0,subset=['RECEIPT_DATE','BOOKED_DATE'])
    df = df[df['RECEIPT_DATE']>df['BOOKED_DATE']] # false
    df = df.set_index('PO_STRING',drop=True)   
    return df

data = treat_null_values(data)
data.shape

(82604, 84)

In [10]:
# optional function
#  This funtion is to analyse the different types of columns # optional
def analyze_data_types(df):
    date_columns_df = df.select_dtypes(include=['datetime64'])
    object_columns_df = df.select_dtypes(include=['object'])
    numerics= [ 'int64','float64']
    numeric_columns_df = df.select_dtypes(include = numerics)
    
    return date_columns_df, object_columns_df,numeric_columns_df

date_df, objects_df,numeric_df=analyze_data_types(data)

In [11]:
def drop_features(df,cols_to_drop):
    df['diff_SO_LINE_FIRST_SO_LINE_LAST'] = df['SO_LINE_LAST'] - df['SO_LINE_FIRST']

    df.drop(cols_to_drop,axis=1,inplace=True)
    
    return df

columns_to_remove = ['ORG_ID','ATTRIBUTE_NUMBER1','ATTRIBUTE_NUMBER2','ATTRIBUTE_NUMBER6','ATTRIBUTE_NUMBER7','ATTRIBUTE_NUMBER8',
                    'UNIT_PRICE_USD','SO_LINE_FIRST','SO_LINE_LAST']

#objects_columns can be removed except 'TRANSPORTATION_LEAD_TIME'

objects_df_cols=list(objects_df.columns)
objects_df_cols.remove('TRANSPORTATION_LEAD_TIME')
cols_to_remove = columns_to_remove + objects_df_cols

data = drop_features(data,cols_to_remove)

In [13]:
data=data.dropna()
data.shape

(60847, 48)

In [14]:
# function to re-engineer the dates
def difference_dates(df,col1,col2):
            
    df['diff_'+col1+col2]=(df[col1]-df[col2]).dt.days
    df.drop([col1,col2],axis=1,inplace = True)
    
    return df



In [16]:
# call function on the required features    
data = difference_dates(data,'RTS_','RTS_FINAL')
data = difference_dates(data,'SO_CWD','SO_CREATION_DATE')
data = difference_dates(data,'EOQ_CONTR','EOQ_REQUEST')
data = difference_dates(data,'CONTRACTUAL_DATE','FULFILLMENT_CONTRACTUAL_DATE')
data = difference_dates(data,'RECEIPT_DATE','PO_FIRST_APPROVED_DATE')  # target variable
#data = difference_dates(data,'RECEIPT_DATE','BOOKED_DATE') 
data = difference_dates(data,'PROMISED_DATE_LAST','PROMISED_DATE_FIRST')

In [17]:
# since most of the rows are same as NEED_BY_DATE','NEED_DATE_FIRST', 'NEED_DATE_LAST'
# 'NEED_BY_DATE' has same values as 'SUP_PO_NEED_BY_DATE'
# instead of these 3 features only one feature can be used and name it 'NEED_DATE_'
# 'DATE_WHEN_NEED_REV'
data['NEED_DATE_'] = data['NEED_BY_DATE']
data.drop(['NEED_DATE_FIRST', 'NEED_DATE_LAST','SUP_PO_NEED_BY_DATE','DATE_WHEN_NEED_REV',
         'NEED_BY_DATE'],axis=1,inplace=True)

# # both SCHEDULE_SHIP_DATE and 'LINE_REQUEST_DATE' are greater than NEED_DATE so take the diff
data['diff_SCHEDULE_SHIP_DATE_NEED_DATE_'] = (data['SCHEDULE_SHIP_DATE']-data['NEED_DATE_']).dt.days
data['diff_LINE_REQUEST_DATE_NEED_DATE_'] = (data['LINE_REQUEST_DATE']-data['NEED_DATE_']).dt.days

data.drop(['LINE_REQUEST_DATE','SCHEDULE_SHIP_DATE','BOOKED_DATE','POR_APPROVED_DATE'],axis=1,inplace=True)

data['NEED_DATE_numeric'] = data['NEED_DATE_'].map(dt.datetime.toordinal)
data.drop('NEED_DATE_',axis=1,inplace=True)
data['TRANSPORTATION_LEAD_TIME'] =data['TRANSPORTATION_LEAD_TIME'].astype(float)

In [18]:
data.shape

(60847, 36)

In [17]:
# if we remove these columns it gives 15 as rmse
# target = 'diff_RECEIPT_DATEPO_FIRST_APPROVED_DATE'
# extra_cols_drop=['VENDOR_ID','LID','SUP_PO_LINE_LOCATION_IDDD','HEADER_ID',
#                      'LINE_ID', 'SHIPMENT_NUMBER']
# extra_cols_df = df[extra_cols_drop]
# df.drop(extra_cols_drop,axis=1,inplace=True)

In [18]:
#extra_cols_df[extra_cols_df.columns].apply(lambda x: x.corr(df[target]))

VENDOR_ID                    0.098714
LID                         -0.382041
SUP_PO_LINE_LOCATION_IDDD   -0.382041
HEADER_ID                   -0.272195
LINE_ID                     -0.285948
SHIPMENT_NUMBER              0.003594
dtype: float64

In [19]:
# make a copy of df
df1=data.copy()

In [20]:
target = 'diff_RECEIPT_DATEPO_FIRST_APPROVED_DATE'

X=df1.drop(target,axis=1)
y=df1[target]
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.2,random_state=4)
print(train_X.shape,test_X.shape,train_y.shape,test_y.shape)

(48677, 35) (12170, 35) (48677,) (12170,)


In [21]:
# Try a simple XGB model
other_params={'learning_rate':0.1,'n_estimators':300,'max_depth':5,'min_child_weight':1,'subsample':0.8,'colsample_bytree':0.8}

regressor =xgb.XGBRegressor(objective='reg:squarederror',**other_params)

xgb_regressor =regressor.fit(train_X, train_y)
predictions = regressor.predict(test_X)

# promised first date 28
# diff beween recived and booked date
print('Root_mean_squared_error_test set_',math.sqrt(mean_squared_error(test_y,predictions)))
print('R2 Score_test set_xgb',r2_score(test_y,predictions))

Root_mean_squared_error_test set_ 12.04829075190926
R2 Score_test set_xgb 0.9672761841028197


# Randomized search with XGB

In [ ]:
# Multi output regressor try this,not tried, try this later
# params={'n_estimators':[100,300,500,700],
#        'min_child_weight':[1,5,9],
#        'gamma':[i/10.0 for i in range(3,6)],
#         'subsample':[i/10.0 for i in range(6,11)],
#         'colsample_bytree':[i/10.0 for i in range(6,11)],
#         'max_depth':[2,4,8],
#         'objective':['reg:squarederror'],
#         'booster':['gbtree'],
#         'eval_metric':['rmse'],
#         'eta':[1/10 for i in range(1,9)],
#        }
        
# reg=xgb.XGBRegressor(tree_method='gpu_hist',gpu_id=0)
# #multioutputregressor_xgb=MultiOutputRegressor(reg)
# n_iter_search=250
# random_search=RandomizedSearchCV(reg,param_distributions=params,n_iter=n_iter_search,
#                                  cv=5,verbose=0,scoring='neg_mean_squared_error')

# start=time.time()
# Randomized_research_Multioutput=MultiOutputRegressor(random_search).fit(train_X,train_y)
# print('RandomizedSearchCV with Multioutput took %.2f seconds for %d candidates'
#       'parameter settings' % ((time.time()-start),n_iter_search))

# APPLY DNN

In [21]:
import tensorflow as tf
#tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from sklearn.model_selection import RandomizedSearchCV


import pickle

In [32]:
# basic dnn

In [22]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [23]:
# func3
from sklearn.preprocessing import MinMaxScaler, StandardScaler
def scale_datasets(x_train, x_test):
    """
    Standard Scale test and train data
    Z - Score normalization
    """
    standard_scaler = StandardScaler()
    x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
    )
    x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
    )
    return x_train_scaled, x_test_scaled
# scale the dataset
X_train_scaled, X_test_scaled = scale_datasets(train_X, test_X)

In [ ]:
# see whether random search is helpful

In [24]:
from tensorflow import keras 
from keras import optimizers

# Apply Random Search on Neural Network

In [25]:
def build_model( nl1=1, nl2=1,  nl3=1,nn1=256, nn2=128, nn3 = 64, lr=0.01, decay=0., l1=0.01, l2=0.01,
                act = 'relu', dropout=0, input_shape=[train_X.shape[1]]):
    
    
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    # for the firt layer we need to specify the input dimensions
    first=True
    
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    model.add(Dense(1))
    model.compile(loss='mae', optimizer= 'adam', metrics=['mae'],)
    return model

In [26]:
# model class to use in the scikit random search CV 
model = KerasRegressor(build_fn=build_model, epochs=50, batch_size=32, verbose=1)

In [27]:
# learning algorithm parameters
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# activation
activation=['relu', 'tanh']

# numbers of layers
nl1 = [0,1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[256,128,64]
nn2=[128,64,32]
nn3=[64,32]

# dropout and regularisation
dropout = [0.05]
l1 = [0.001,0.0001]
l2 = [0.001,0.0001]

# dictionary summary
param_grid = dict(
                    nl1=nl1, nl2=nl2, nl3=nl3, nn1=nn1, nn2=nn2, nn3=nn3,
                    act=activation, l1=l1, l2=l2, lr=lr, decay=decay, dropout=dropout, 
                    input_shape=[X.shape[1]]
                 )

In [28]:
random_nn_model = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=20)

In [29]:
import time
start_time =time.time()
random_result = random_nn_model.fit(X_train_scaled, train_y)
print(f'Time taken is {time.time()-start_time} ')

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START act=tanh, decay=0, dropout=0.05, input_shape=35, l1=0.001, l2=0.0001, lr=0.001, nl1=0, nl2=2, nl3=3, nn1=256, nn2=64, nn3=64


2022-06-03 05:42:21.131328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31009 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0
2022-06-03 05:42:21.912740: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
1015/1015 [==============================] - 5s 3ms/step - loss: 58.9986 - mae: 57.5515
Epoch 2/50
1015/1015 [==============================] - 3s 3ms/step - loss: 29.5599 - mae: 28.4948
Epoch 3/50
1015/1015 [==============================] - 3s 3ms/step - loss: 19.6261 - mae: 18.6466
Epoch 4/50
1015/1015 [==============================] - 3s 3ms/step - loss: 15.7913 - mae: 14.8844
Epoch 5/50
1015/1015 [==============================] - 4s 4ms/step - loss: 13.8097 - mae: 12.9586
Epoch 6/50
1015/1015 [==============================] - 3s 3ms/step - loss: 12.6967 - mae: 11.8928
Epoch 7/50
1015/1015 [==============================] - 3s 3ms/step - loss: 12.1045 - mae: 11.3426
Epoch 8/50
1015/1015 [==============================] - 3s 3ms/step - loss: 11.7925 - mae: 11.0635
Epoch 9/50
1015/1015 [==============================] - 4s 3ms/step - loss: 11.4126 - mae: 10.7142
Epoch 10/50
1015/1015 [==============================] - 3s 3ms/step - loss: 11.2853 - mae: 10.6054
Epoch 11/

1015/1015 [==============================] - 3s 3ms/step - loss: 9.8668 - mae: 9.3641
Epoch 30/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.8412 - mae: 9.3341
Epoch 31/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.8530 - mae: 9.3480
Epoch 32/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.8115 - mae: 9.3121
Epoch 33/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.8126 - mae: 9.3129
Epoch 34/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.7795 - mae: 9.2741
Epoch 35/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.7215 - mae: 9.2242
Epoch 36/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.7904 - mae: 9.2868
Epoch 37/50
1015/1015 [==============================] - 4s 3ms/step - loss: 9.6765 - mae: 9.1807
Epoch 38/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.7024 - mae: 9.2029
Epoch 39/50
1015/1015 [=========

Epoch 4/50
1015/1015 [==============================] - 3s 3ms/step - loss: 8.6066 - mae: 8.4594
Epoch 5/50
1015/1015 [==============================] - 3s 3ms/step - loss: 8.1527 - mae: 8.0063
Epoch 6/50
1015/1015 [==============================] - 3s 3ms/step - loss: 7.8546 - mae: 7.7076
Epoch 7/50
1015/1015 [==============================] - 3s 3ms/step - loss: 7.4750 - mae: 7.3273
Epoch 8/50
1015/1015 [==============================] - 3s 3ms/step - loss: 7.4301 - mae: 7.2827
Epoch 9/50
1015/1015 [==============================] - 3s 3ms/step - loss: 7.3142 - mae: 7.1663
Epoch 10/50
1015/1015 [==============================] - 3s 3ms/step - loss: 7.2066 - mae: 7.0594
Epoch 11/50
1015/1015 [==============================] - 3s 3ms/step - loss: 7.1262 - mae: 6.9785
Epoch 12/50
1015/1015 [==============================] - 3s 3ms/step - loss: 6.9918 - mae: 6.8443
Epoch 13/50
1015/1015 [==============================] - 3s 3ms/step - loss: 7.0105 - mae: 6.8631
Epoch 14/50
1015/1015 [===

...

1015/1015 [==============================] - 4s 3ms/step - loss: 10.0260 - mae: 9.5219
Epoch 30/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.9889 - mae: 9.4823
Epoch 31/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.9907 - mae: 9.4867
Epoch 32/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.9590 - mae: 9.4490
Epoch 33/50
1015/1015 [==============================] - 4s 4ms/step - loss: 9.9690 - mae: 9.4582
Epoch 34/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.9664 - mae: 9.4604
Epoch 35/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.9934 - mae: 9.4876
Epoch 36/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.8962 - mae: 9.3875
Epoch 37/50
1015/1015 [==============================] - 3s 3ms/step - loss: 9.8178 - mae: 9.3129
Epoch 38/50
1015/1015 [==============================] - 4s 4ms/step - loss: 9.8214 - mae: 9.3170
Epoch 39/50
1015/1015 [========

1015/1015 [==============================] - 3s 3ms/step - loss: 25.3509 - mae: 24.8801
Epoch 4/50
1015/1015 [==============================] - 4s 4ms/step - loss: 17.0238 - mae: 16.5724
Epoch 5/50
1015/1015 [==============================] - 3s 3ms/step - loss: 14.2282 - mae: 13.7916
Epoch 6/50
1015/1015 [==============================] - 4s 4ms/step - loss: 12.8535 - mae: 12.4185
Epoch 7/50
1015/1015 [==============================] - 3s 3ms/step - loss: 12.0966 - mae: 11.6624
Epoch 8/50
1015/1015 [==============================] - 3s 3ms/step - loss: 11.6606 - mae: 11.2292
Epoch 9/50
1015/1015 [==============================] - 4s 3ms/step - loss: 11.3163 - mae: 10.8818
Epoch 10/50
1015/1015 [==============================] - 4s 3ms/step - loss: 11.1385 - mae: 10.7022
Epoch 11/50
1015/1015 [==============================] - 3s 3ms/step - loss: 10.9208 - mae: 10.4798
Epoch 12/50
1015/1015 [==============================] - 3s 3ms/step - loss: 10.7994 - mae: 10.3608
Epoch 13/50
1015/1

1015/1015 [==============================] - 4s 3ms/step - loss: 7.1336 - mae: 6.9481
Epoch 33/50
1015/1015 [==============================] - 4s 3ms/step - loss: 6.9760 - mae: 6.7894
Epoch 34/50
1015/1015 [==============================] - 4s 4ms/step - loss: 6.9286 - mae: 6.7416
Epoch 35/50
1015/1015 [==============================] - 4s 4ms/step - loss: 6.9311 - mae: 6.7440
Epoch 36/50
1015/1015 [==============================] - 4s 4ms/step - loss: 6.8938 - mae: 6.7059
Epoch 37/50
1015/1015 [==============================] - 4s 4ms/step - loss: 6.9085 - mae: 6.7198
Epoch 38/50
1015/1015 [==============================] - 4s 4ms/step - loss: 6.8796 - mae: 6.6902
Epoch 39/50
1015/1015 [==============================] - 4s 3ms/step - loss: 6.8347 - mae: 6.6443
Epoch 40/50
1015/1015 [==============================] - 3s 3ms/step - loss: 6.8069 - mae: 6.6165
Epoch 41/50
1015/1015 [==============================] - 4s 3ms/step - loss: 6.7574 - mae: 6.5666
Epoch 42/50
1015/1015 [=========

1015/1015 [==============================] - 4s 4ms/step - loss: 9.2150 - mae: 9.0533
Epoch 8/50
1015/1015 [==============================] - 3s 3ms/step - loss: 8.9389 - mae: 8.7756
Epoch 9/50
1015/1015 [==============================] - 4s 4ms/step - loss: 8.6655 - mae: 8.5015
Epoch 10/50
1015/1015 [==============================] - 4s 3ms/step - loss: 8.6172 - mae: 8.4525
Epoch 11/50
1015/1015 [==============================] - 4s 4ms/step - loss: 8.3810 - mae: 8.2161
Epoch 12/50
1015/1015 [==============================] - 4s 3ms/step - loss: 8.2688 - mae: 8.1028
Epoch 13/50
1015/1015 [==============================] - 3s 3ms/step - loss: 8.1934 - mae: 8.0266
Epoch 14/50
1015/1015 [==============================] - 4s 4ms/step - loss: 8.0783 - mae: 7.9109
Epoch 15/50
1015/1015 [==============================] - 3s 3ms/step - loss: 8.1102 - mae: 7.9421
Epoch 16/50
1015/1015 [==============================] - 3s 3ms/step - loss: 8.0063 - mae: 7.8373
Epoch 17/50
1015/1015 [===========

1522/1522 [==============================] - 4s 3ms/step - loss: 5.7413 - mae: 5.5758
Epoch 39/50
1522/1522 [==============================] - 5s 3ms/step - loss: 5.7597 - mae: 5.5936
Epoch 40/50
1522/1522 [==============================] - 5s 3ms/step - loss: 5.6990 - mae: 5.5333
Epoch 41/50
1522/1522 [==============================] - 4s 3ms/step - loss: 5.7422 - mae: 5.5768
Epoch 42/50
1522/1522 [==============================] - 4s 3ms/step - loss: 5.7200 - mae: 5.5543
Epoch 43/50
1522/1522 [==============================] - 4s 3ms/step - loss: 5.6208 - mae: 5.4548
Epoch 44/50
1522/1522 [==============================] - 5s 3ms/step - loss: 5.6393 - mae: 5.4725
Epoch 45/50
1522/1522 [==============================] - 5s 3ms/step - loss: 5.6180 - mae: 5.4508
Epoch 46/50
1522/1522 [==============================] - 5s 3ms/step - loss: 5.6594 - mae: 5.4920
Epoch 47/50
1522/1522 [==============================] - 4s 3ms/step - loss: 5.6030 - mae: 5.4350
Epoch 48/50
1522/1522 [=========

In [30]:
# for 50 epochs
best_model_50e = random_result.best_estimator_
print(random_result.best_params_)

{'nn3': 64, 'nn2': 128, 'nn1': 64, 'nl3': 1, 'nl2': 0, 'nl1': 3, 'lr': 0.0001, 'l2': 0.0001, 'l1': 0.0001, 'input_shape': 35, 'dropout': 0.05, 'decay': 1e-06, 'act': 'relu'}


In [31]:
#  50 epochs works better than 10 epochs
predictions_rs_nn_50 = best_model_50e.predict(X_test_scaled)

print("RMSE Test_x",np.sqrt(mean_squared_error(test_y,predictions_rs_nn_50))) 
print("R2 score Test_x",r2_score(test_y,predictions_rs_nn_50))

381/381 [==============================] - 1s 1ms/step
RMSE Test_x 8.906044093954996
R2 score Test_x 0.9821193729965048


In [33]:
# for 10 epochs
def eval_metrics(df,col_real, col_pred):
    print('accuracy 3days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 3) / results_df[col_real].shape[0])
    print('accuracy 5days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 5) / results_df[col_real].shape[0])
    print('accuracy 7days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 7) / results_df[col_real].shape[0])
    print('accuracy 10days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 10) / results_df[col_real].shape[0])
    print('accuracy 20days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 20) / results_df[col_real].shape[0])
    print('accuracy 30days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 30) / results_df[col_real].shape[0])
    print('accuracy 90days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 90) / results_df[col_real].shape[0])

    print('************************************************************************************************')
    print('accuracy unpaired 30days', np.sum( ( (results_df[col_real] - results_df[col_pred])) <= 30) / results_df[col_real].shape[0])    
    
    return 



results_df = pd.DataFrame({'original_diff':test_y,'predicted_diff':predictions_rs_nn_50})
eval_metrics(results_df,'original_diff','predicted_diff')

accuracy 3days 0.6359901396877568
accuracy 5days 0.8221035332785538
accuracy 7days 0.9109285127362367
accuracy 10days 0.9585866885784716
accuracy 20days 0.9916187345932621
accuracy 30days 0.9960558751027115
accuracy 90days 0.9990139687756779
************************************************************************************************
accuracy unpaired 30days 0.999671322925226
